In [3]:
# --- CONFIGURATION ---
# Optional: Filter by a specific workspace ID
WORKSPACE_ID = "" # e.g., "b4c5..."

# Construct the command arguments
args = []
if WORKSPACE_ID:
    args.append(f"--workspace {WORKSPACE_ID}")

args_str = " ".join(args)
print(f"🚀 Running extraction with args: {args_str}")

🚀 Running extraction with args: 


In [4]:
import os
import sys

# Ensure we are in the project root or can find the script
# Adjust this path if necessary to point to your project root
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
script_path = os.path.join(project_root, "src/usf_fabric_monitoring/scripts/extract_fabric_item_details.py")

print(f"📂 Project Root: {project_root}")
print(f"📜 Script Path: {script_path}")

if not os.path.exists(script_path):
    print("❌ Error: Script not found. Please check the path.")
else:
    print("✅ Script found.")

📂 Project Root: /home/sanmi/Documents/J'TOYE_DIGITAL/LEIT_TEKSYSTEMS/1_Project_Rhico/usf_fabric_monitoring
📜 Script Path: /home/sanmi/Documents/J'TOYE_DIGITAL/LEIT_TEKSYSTEMS/1_Project_Rhico/usf_fabric_monitoring/src/usf_fabric_monitoring/scripts/extract_fabric_item_details.py
✅ Script found.


In [5]:
# --- CONFIGURATION ---
# Set to True to fetch ALL workspaces in the tenant (Requires Admin API permissions)
TENANT_WIDE = False 

# Optional: Filter by a specific workspace ID
WORKSPACE_ID = "" # e.g., "b4c5..."

# Construct the command arguments
args = []
if TENANT_WIDE:
    args.append("--tenant-wide")
if WORKSPACE_ID:
    args.append(f"--workspace {WORKSPACE_ID}")

args_str = " ".join(args)
print(f"🚀 Running extraction with args: {args_str}")

🚀 Running extraction with args: 


In [6]:
# Execute the script
# We use !python to run it as a subprocess, ensuring it runs exactly as it would from the terminal.
# We pass the PYTHONPATH to ensure it can find the 'usf_fabric_monitoring' package.

command = f"export PYTHONPATH={project_root}/src && python {script_path} {args_str}"
print(f"Executing: {command}")

!{command}

Executing: export PYTHONPATH=/home/sanmi/Documents/J'TOYE_DIGITAL/LEIT_TEKSYSTEMS/1_Project_Rhico/usf_fabric_monitoring/src && python /home/sanmi/Documents/J'TOYE_DIGITAL/LEIT_TEKSYSTEMS/1_Project_Rhico/usf_fabric_monitoring/src/usf_fabric_monitoring/scripts/extract_fabric_item_details.py 


In [ ]:
# 2. Execute Extraction using Package Logic (v0.1.2+)
# This uses the new reusable module that powers the main pipeline.

from usf_fabric_monitoring.scripts.extract_fabric_item_details import run_item_details_extraction
import logging

# --- CONFIGURATION ---
# Optional: Filter by a specific workspace ID
WORKSPACE_ID = None # e.g., "b4c5..."
OUTPUT_DIR = "/lakehouse/default/Files/exports/fabric_item_details" # Adjust for Fabric

print(f"🚀 Starting extraction to {OUTPUT_DIR}...")

result = run_item_details_extraction(
    workspace_id=WORKSPACE_ID,
    output_dir=OUTPUT_DIR,
    log_level="INFO"
)

if result["status"] == "success":
    print(f"✅ Extraction Complete!")
    print(f"   • Jobs Saved: {result.get('jobs_count', 0)}")
    print(f"   • Lakehouses Scanned: {result.get('lakehouses_count', 0)}")
    print(f"   • Files: {result.get('files', [])}")
else:
    print(f"❌ Extraction Failed: {result.get('message')}")

🔍 Fetching workspaces (Tenant Wide: False)...
Found 3 workspaces.
Found 3 workspaces.


Job instances endpoint not found for item b51cda97-b711-453f-ad04-dc2a901f8684
Job instances endpoint not found for item 3cf46268-c3aa-458f-a0b9-c1ba00d860d3
Job instances endpoint not found for item 3cf46268-c3aa-458f-a0b9-c1ba00d860d3
Job instances endpoint not found for item 22cc64a4-046b-4491-b891-bcab4b28474f
Job instances endpoint not found for item 22cc64a4-046b-4491-b891-bcab4b28474f
Job instances endpoint not found for item b0a472c3-9918-4880-b292-43e36b5c182d
Job instances endpoint not found for item b0a472c3-9918-4880-b292-43e36b5c182d
Job instances endpoint not found for item 38b77fd4-75dd-43cd-9009-96ee786a56ba
Job instances endpoint not found for item 38b77fd4-75dd-43cd-9009-96ee786a56ba
Job instances endpoint not found for item f3304b39-5d5b-4483-bd84-0e21211f620d
Job instances endpoint not found for item f3304b39-5d5b-4483-bd84-0e21211f620d
Job instances endpoint not found for item d0c97ac5-958f-48f0-a3ef-5095c7329af7
Job instances endpoint not found for item d0c97ac5-9

KeyboardInterrupt: 

: 

In [ ]:
# 4. Define Schema & Load Data for Analysis
from pyspark.sql.types import StructType, StructField, StringType, DoubleType
from pyspark.sql.functions import col, to_timestamp, count, desc, avg, max, min

# Schema including End Time and nested Failure Reason
json_schema = StructType([
    StructField("id", StringType(), True),
    StructField("itemId", StringType(), True),
    StructField("jobType", StringType(), True),
    StructField("invokeType", StringType(), True),
    StructField("status", StringType(), True),
    StructField("failureReason", StructType([
        StructField("errorCode", StringType(), True),
        StructField("message", StringType(), True),
        StructField("requestId", StringType(), True)
    ]), True),
    StructField("rootActivityId", StringType(), True),
    StructField("startTimeUtc", StringType(), True),
    StructField("endTimeUtc", StringType(), True),
    StructField("_workspace_name", StringType(), True),
    StructField("_item_name", StringType(), True),
    StructField("_item_type", StringType(), True),
    StructField("duration", DoubleType(), True)
])

# Load the just-saved files (or all files in the directory)
try:
    # Updated to read the new generic 'jobs_*.json' pattern
    # Note: Ensure ONELAKE_EXPORT_PATH is defined or use the OUTPUT_DIR from above
    path_to_read = f"{OUTPUT_DIR}/jobs_*.json"
    print(f"📂 Reading from: {path_to_read}")
    
    df = spark.read.option("multiLine", "true").schema(json_schema).json(path_to_read)
    
    # Transform
    df_processed = df.withColumn("start_time", to_timestamp(col("startTimeUtc"))) \
                     .withColumn("end_time", to_timestamp(col("endTimeUtc"))) \
                     .withColumn("error_code", col("failureReason.errorCode")) \
                     .withColumn("error_message", col("failureReason.message"))

    print(f"📊 Loaded {df_processed.count()} records for analysis.")
except Exception as e:
    print(f"⚠️ No data loaded: {e}")
    df_processed = None

In [ ]:
# 5. Generate Insights

if df_processed:
    print("=== Recent Runs (Detailed) ===")
    df_processed.orderBy(col("start_time").desc()).limit(10) \
        .select(
            col("_workspace_name").alias("Workspace"),
            col("_item_name").alias("Item"),
            col("status").alias("Status"),
            col("start_time").alias("Start Time"),
            col("end_time").alias("End Time"),
            col("duration").alias("Duration(s)"),
            col("error_code").alias("Error")
        ).show(truncate=False)

    print("\n=== Failure Analysis by Item Type ===")
    df_processed.filter(col("status") == "Failed") \
        .groupBy("_item_type", "error_code") \
        .agg(count("*").alias("Failures")) \
        .orderBy(desc("Failures")) \
        .show(truncate=False)

    print("\n=== Duration Stats (Successful Runs) ===")
    df_processed.filter(col("status") == "Completed") \
        .groupBy("_item_type") \
        .agg(
            avg("duration").alias("Avg Duration"),
            max("duration").alias("Max Duration"),
            min("duration").alias("Min Duration")
        ).show()
        
    print("\n=== Note on User Attribution ===")
    print("The 'User' or 'Principal' field is currently unavailable in the Fabric Job Instance API.")
    print("To track who triggered a run, consider using Audit Logs (ActivityEvents) instead of Item Job Instances.")